Будем использовать реализацию Word2vec в библиотеке **Gensim**, а в качестве предобученных моделей возьмем модели Андрея Кутузова и Лизы Кузьменко с сайта [RusVectōrēs.](https://rusvectores.org/ru/models/). 

In [1]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors

В качестве моделей давайте возьмем 

1) araneum_none_fasttextcbow_300_5_2018 (fasttext) - модель, обученная на интернет-корпусе русского языка


2) ruscorpora_upos_skipgram_300_5_2018 (word2vec) - модель, обученная НКРЯ

## word2vec + fasttext

Существуют несколько форматов, в которых могут храниться модели - .vec и .model 

1) Первый формат считается классическим вариантом модели word2vec. Для загрузки таакой модели надо воспользоваться методом *KeyedVectors.load_word2vec_format*. 
Модель может быть бинарной, для ее загрузки надо передать параметр binary, равный True. 

2) Формат .model - собственный формат gensim. Такую модель надо загружать с помощью метода *KeyedVectors.load*.

 **1) если модель без тэгов**

In [180]:
# загрузка модели

model_file = '../../data/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)


#проверка наличия слова в словаре

lemma = 'заграница'
lemma in model

True

**2) если модель с POS-тэггингом**

In [178]:
# загрузка модели

# model_file = '../../data/ruscorpora_upos_skipgram_300_5_2018.vec'
# model_POS = KeyedVectors.load_word2vec_format(model_file, binary=False)


#проверка наличия слова в словаре

lemma = 'заграница_NOUN'
lemma in model

True

**3) получение вектора слова**

In [179]:
lemma = 'заграница'
lemma in model

True

In [184]:
v1 = model['заграница']
v2 = model_POS.wv['заграница_NOUN']

(v1 == v2).all()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


False

## Получение вектора документа

Отлично, вектора для слов получены. Что с ними делать дальше? 

Есть два подхода (а точнее есть один, а второй мы придумали, потому что с одним жить нельзя).
> Классика - для получения вектора документа нужно взять и усреднить все вектора его слов
 
$$ vec_{doc} = \frac {\sum_{i=0}^{n} vec_i}{len(d)} $$


In [187]:
import numpy as np

# сделали препроцессинг, получили леммы 
lemmas = ['старинный', 'замок']

# создаем вектор-маску
lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
vec = np.zeros((model.vector_size,))

# если слово есть в модели, берем его вектор
for idx, lemma in enumerate(lemmas):
    if lemma in model:
        lemmas_vectors[idx] = model[lemma]
        
# проверка на случай, если на вход пришел пустой массив
if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)


> Эксперимент - представим документ не в виде одного уредненного вектора, а как матрицу векторов входящих в него слов

```
 слово1 |  v1_300
 слово2 |  v2_300
 слово3 |  v3_300
 слово4 |  v4_300
```

> Отлично, теперь каждый документ представлен в виде матрицы векторов своих слов. Но нам надо получить близость матрицы документа в коллекции и матрицы входящего запроса. Как? Умножим две матрицы друг на друга - одна матрица размером d x 300, другая q x 300 - получим попарную близость слов из каждого документа - матрицу размером d x q.


In [163]:
# возьмем игрушечный пример кейса

text1 = 'турция' 
text2 = 'нужна справка срочно'
query = 'быстрая справка'

In [166]:
# построим матрицы всех документов

def create_doc_matrix(text):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        if lemma in model.wv:
            lemmas_vectors[idx] = normalize_vec(model.wv[lemma])
            
    return lemmas_vectors    


text1_m = create_doc_matrix(text1)
text2_m = create_doc_matrix(text2)
query_m = create_doc_matrix(query)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [167]:
# размер матрицы как и ожидали
query_m.shape

(2, 300)

In [193]:
# посмотрим на близость слов первого текста и слов запроса
text1_m.dot(query_m.T)

array([[0.09587915, 0.01183069]])

In [159]:
# посмотрим на близость слов второго текста и слов запроса
text2_m.dot(query_m.T)

array([[-0.0260624 ,  0.11607588],
       [ 0.01341236,  1.00000011],
       [ 0.22505549,  0.33582122]])

In [194]:
docs_m = [text1_m, text2_m]

    
def search(docs, query, reduce_func=np.max, axis=0):
    sims = []
    for doc in docs:
        sim = doc.dot(query.T)
        sim = reduce_func(sim, axis=axis)
        sims.append(sim.sum())
    print(sims)
    return np.argmax(sims)


search(docs_m, query_m)

[0.10770983955697251, 1.225055597288777]


1

## Задание

Реализуйте поиск по нашему стандартному Covid корпусу с помощью модели на Araneum двумя способами:

    1. преобразуйте каждый документ в вектор через усреднение векторов его слов и реализуйте поисковик как 
    обычно через умножение матрицы документов коллекции на вектор запроса 
    2. экспериментальный способ - реализуйте поиск ближайшего документа в коллекции к запросу, преобразовав 
    каждый документ в матрицу (количество слов x размер модели)
    
Посчитайте качество поиска для каждой модели на тех же данных, что и в предыдущем задании. В качестве препроцессинга используйте две версии - с удалением NER и без удаления.  

In [1]:
import re
import pandas as pd
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
Загрузка моделей и стопслов

In [2]:
def get_model():
    global model
    model_file = 'model_files/araneum_none_fasttextcbow_300_5_2018.model'
    model = KeyedVectors.load(model_file)
    
get_model()
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [3]:
%%time
import warnings
warnings.filterwarnings('ignore')

from deeppavlov import configs, build_model
ner_model = build_model(configs.ner.ner_rus)

[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>
[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>
[nltk_data] Error loading perluniprops: <urlopen error [Errno 8]
[nltk_data]     nodename nor servname provided, or not known>
[nltk_data] Error loading nonbreaking_prefixes: <urlopen error [Errno
[nltk_data]     8] nodename nor servname provided, or not known>
2020-10-21 19:28:07.663 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /Users/elizavetaersova/.deeppavlov/models/ner_rus/word.dict]
2020-10-21 19:28:07.721 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /Users/elizavetaersova/.deeppavlov/models/ner_rus/tag.dict]
2020-10-21 19:28:07.726 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /Users/








Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.cast` instead.



2020-10-21 19:28:20.26 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


2020-10-21 19:28:22.429 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-10-21 19:28:25.124 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /Users/elizavetaersova/.deeppavlov/models/ner_rus/model]



INFO:tensorflow:Restoring parameters from /Users/elizavetaersova/.deeppavlov/models/ner_rus/model
CPU times: user 8.6 s, sys: 3.05 s, total: 11.6 s
Wall time: 19.4 s


In [ ]:
Индексирование базы

In [ ]:
Загружу и проиндексирую тексты в разных вариантах предобработки

In [12]:
queries_base = pd.read_csv('X_train_NER_preproc.csv')
tokens_corpus = queries_base.clean_text.tolist()

In [5]:
def preprocess(text):
    text = text.lower()
    text = re.sub('[^а-яё]', ' ', text)
    text = [token for token in text.split() if token not in russian_stopwords]
    text = ' '.join(text)
    return text

def normalize_vec(v):
    return v / np.sqrt(np.sum(v ** 2))

def preprocess_with_deepmipt(text: str) -> str:
    anno = ner_model([text])
    tokens = anno[0][0]
    tags = anno[1][0]
    clean_tokens = []
    for token, tag in zip(tokens, tags):
        if tag == 'O':
            clean_tokens.append(token)
    clean_text = ' '.join(clean_tokens)
    return clean_text

def query_base_indexing_w2v_basic():
    global w2v_basic_matrix
    vectors = []
    for query in tokens_corpus:
        tokens = query.split()
        tokens_vectors = np.zeros((len(tokens), model.vector_size))
        vec = np.zeros((model.vector_size,))
        for idx, token in enumerate(tokens):
            if token in model:
                tokens_vectors[idx] = model[token]
        if tokens_vectors.shape[0] is not 0:
            vec = np.mean(tokens_vectors, axis=0)
        vec = normalize_vec(vec)
        vectors.append(vec)
    w2v_basic_matrix = np.matrix(vectors)


def w2v_advanced_index_single_doc(text):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))

    for idx, lemma in enumerate(lemmas):
        if lemma in model:
            lemmas_vectors[idx] = normalize_vec(model[lemma])
    return lemmas_vectors


def query_base_indexing_w2v_advanced():
    global w2v_advanced_index
    w2v_advanced_index = []
    for query in tokens_corpus:
        query_matrix = w2v_advanced_index_single_doc(query)
        w2v_advanced_index.append(query_matrix)
    return w2v_advanced_index

In [13]:
%%time
query_base_indexing_w2v_basic()
query_base_indexing_w2v_advanced()

CPU times: user 5.19 s, sys: 197 ms, total: 5.39 s
Wall time: 5.45 s


In [7]:
def w2v_basic_search(query):
    tokens = preprocess(query).split()
    tokens_vectors = np.zeros((len(tokens), model.vector_size))
    vec = np.zeros((model.vector_size,))
    for idx, token in enumerate(tokens):
        if token in model:
            tokens_vectors[idx] = model[token]
    if tokens_vectors.shape[0] is not 0:
        vec = np.mean(tokens_vectors, axis=0)
    vec = normalize_vec(vec)
    query_matrix = np.matrix(vec)
    cosine_similarities = linear_kernel(query_matrix, w2v_basic_matrix).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-5:-1]
    answer_doc = related_docs_indices[0]
    answer_id = queries_base['answer_id'][answer_doc]
    return answer_id


def w2v_advanced_search(query):
    query = preprocess(query)
    query_matrix = w2v_advanced_index_single_doc(query)
    sims = []
    for doc in w2v_advanced_index:
        sim = doc.dot(query_matrix.T)
        sim = np.max(sim, axis=0)
        sims.append(sim.sum())
    answer_doc = np.argmax(sims)
    answer_id = queries_base['answer_id'][answer_doc]
    return answer_id


In [ ]:
Измерю качество поиска с удалением NE

In [9]:
X_test = pd.read_csv('X_test_3ner.csv')
y_test = X_test.answer_id.tolist()
X_test_basic = X_test.clean_text.tolist()
X_test_ner_deepmipt = X_test.Deepmipt_NER_removal.tolist()

In [ ]:
Стандартный метод

In [10]:
%%time
n_corr = 0
n_queries = len(X_test_ner_deepmipt)
for query, answer in zip(X_test_ner_deepmipt, y_test):
    pred_answer = w2v_basic_search(query)
    real_answer = answer
    if pred_answer == real_answer:
        n_corr += 1
print(f'Accuracy {n_corr/n_queries}')

Accuracy 0.5434027777777778
CPU times: user 2.9 s, sys: 216 ms, total: 3.12 s
Wall time: 1.68 s


In [ ]:
Экспериментальный метод

In [11]:
%%time
n_corr = 0
n_queries = len(X_test_ner_deepmipt)
for query, answer in zip(X_test_ner_deepmipt, y_test):
    pred_answer = w2v_advanced_search(query)
    real_answer = answer
    if pred_answer == real_answer:
        n_corr += 1
print(f'Accuracy {n_corr/n_queries}')

Accuracy 0.5
CPU times: user 1min 41s, sys: 5.13 s, total: 1min 46s
Wall time: 54.9 s


In [ ]:
Измерю качество поиска без удаления NE

In [ ]:
Стандартный метод

In [14]:
%%time
n_corr = 0
n_queries = len(X_test_basic)
for query, answer in zip(X_test_basic, y_test):
    pred_answer = w2v_basic_search(query)
    real_answer = answer
    if pred_answer == real_answer:
        n_corr += 1
print(f'Accuracy {n_corr/n_queries}')

Accuracy 0.5625
CPU times: user 3.04 s, sys: 223 ms, total: 3.27 s
Wall time: 1.67 s


In [ ]:
Экспериментальный метод

In [15]:
%%time
n_corr = 0
n_queries = len(X_test_basic)
for query, answer in zip(X_test_basic, y_test):
    pred_answer = w2v_advanced_search(query)
    real_answer = answer
    if pred_answer == real_answer:
        n_corr += 1
print(f'Accuracy {n_corr/n_queries}')

Accuracy 0.5138888888888888
CPU times: user 1min 53s, sys: 5.16 s, total: 1min 58s
Wall time: 1min 2s


In [ ]:
Вывод:
    - удаление NE не улучшает качество поиска
    - на тестовой выборке стандартный метод работает лучше